In [ ]:
onnx_model_path = "./model/model.onnx"

ExecutionProvider="QNNExecutionProvider"
transpose = False
if ExecutionProvider == "OpenVINOExecutionProvider":
    onnx_model_path = "./model/ov_model_st_quant.onnx"
elif ExecutionProvider == "VitisAIExecutionProvider":
    transpose = True

In [ ]:
import requests
from PIL import Image
url = "https://onnxruntime.ai/images/dog.jpeg"
response = requests.get(url)
# Save the image to a file
with open("dog.jpeg", "wb") as file:
    file.write(response.content)
img = Image.open("dog.jpeg")
img

In [ ]:
import onnxruntime as ort
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models.resnet import ResNet50_Weights

image_file_path = "dog.jpeg"

# Create ONNX runtime session
def add_ep_for_device(session_options, ep_name, device_type, ep_options=None):
    ep_devices = ort.get_ep_devices()
    for ep_device in ep_devices:
        if ep_device.ep_name == ep_name and ep_device.device.type == device_type:
            print(f"Adding {ep_name} for {device_type}")
            session_options.add_provider_for_devices([ep_device], {} if ep_options is None else ep_options)


session_options = ort.SessionOptions()

add_ep_for_device(session_options, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

session = ort.InferenceSession(
    onnx_model_path, # a model wirh QNN EPContext nodes
    sess_options=session_options,
)

print("Available providers:", session.get_providers())
print("Current provider:", session.get_provider_options())

# Read and preprocess image
image = Image.open(image_file_path)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(image)
if transpose:
    input_tensor = input_tensor.permute(1, 2, 0)
input_batch = input_tensor.unsqueeze(0)

# Run inference
ort_inputs = {session.get_inputs()[0].name: input_batch.numpy()}
ort_outputs = session.run(None, ort_inputs)

# Postprocess to get softmax vector
output = ort_outputs[0]
softmax = torch.nn.functional.softmax(torch.tensor(output), dim=1)

# Extract top 10 predicted classes
top10 = torch.topk(softmax, 10)

# Get label mapping
weights = ResNet50_Weights.DEFAULT
labels = weights.meta["categories"]

# Print results to console
print("Top 10 predictions for ResNet50 v2...")
print("--------------------------------------------------------------")
for i in range(10):
    print(f"Label: {labels[top10.indices[0][i]]}, Confidence: {top10.values[0][i].item():.4f}")
